## Reenforcement Learing: Rock-Paper-Scissors

In [89]:
# TODO: matrix of all possible plays (3x3)
# TODO: way to represent state (previous plays)

# TODO: way to impute state into game


# TODO: reenforcement loop
# action
# calculate
# reward vs no reward
# make decision how to update computer player on reward



def points(player_move, opponent_move):
    if player_move == opponent_move:
        return 0.5
    if (player_move == 'rock' and opponent_move == 'scissors'):
        return 1.0
    if player_move == 'scissors' and opponent_move == 'paper':
        return 1.0
    if player_move == 'paper' and opponent_move == 'rock':
        return 1.0
    return 0.0

### Simulate how a human plays

In [110]:
import random
import numpy as np  

OPTIONS = ["rock", "paper", "scissors"]


# basic rules of the game
def beats_move(move):
    if move == "rock":
        return "paper"
    if move == "paper":
        return "scissors"
    if move == "scissors":
        return "rock"
    return None

# create the weighted probability vector
def biased_move_probabilities(ind, bias):
    vec = [0.0, 0.0, 0.0]
    for (i, v) in enumerate(vec):
        if i == ind:
            vec[i] = 1 / 3 + bias
        else:
            vec[i] = 1 / 3 - bias / 2
    return vec

# determine how a player will play based on their last move and the last move of their opponent
def human_player(last_move, last_opponent_move, bias = 0.67):
    points_last = points(last_move, last_opponent_move)
    if points_last == 1: # player won last round
        ind = OPTIONS.index(last_move) # more likely to play the same move
    elif points_last == 0: # player lost last round
        ind = OPTIONS.index(beats_move(last_opponent_move)) # more likely to play the move that beats the opponent's last move
    else: # player tied last round
        ind = OPTIONS.index(last_move)
        bias = -bias / 2 # player is less likely to play the same move

    wv = biased_move_probabilities(ind, bias)
    # print(wv)
    return random.choices(OPTIONS, weights = wv)[0]

def computer_player(last_move, last_opponent_move, M):
    # assume M is a 3x3 matrix of 3-dimensional probability vectors -> 3 x 3 x 3 = 27 dimensional parameter space
    ind_human = OPTIONS.index(last_opponent_move)
    int_computer = OPTIONS.index(last_move)
    prob_vec = M[ind_human, int_computer]
    return random.choices(OPTIONS, weights = prob_vec)[0]






def play_games(n_games, M):
    human_move = random.choice(OPTIONS)
    computer_move = random.choice(OPTIONS)
    score = 0
    for i in range(n_games):
        new_human_move = human_player(human_move, computer_move)
        new_computer_move = computer_player(computer_move, human_move, M)
        score += points(new_computer_move, new_human_move)
        human_move = new_human_move
        computer_move = new_computer_move

    return score / n_games

def update_M(M, sd = 0.1):
    ind0 = random.choices(range(3))[0]
    ind1 = random.choices(range(3))[0]

    v = M[ind0, ind1]
    v = [x + np.random.normal(0, sd) for x in v] # this is what fixed the problem
    v = [max(0, min(1, x)) for x in v]
    v = [x / sum(v) for x in v]

    M[ind0, ind1] = v

    return M

M = np.random.uniform(0, 1, (3, 3, 3))
M = M / np.sum(M, axis = 2)[:, :, None]

# print(M)
# print(M[0, 2])

# computer_player("rock", "paper", M)

## Do Learning!

In [118]:

import random
import numpy as np

OPTIONS = ["rock", "paper", "scissors"]

POINTS_PER_WIN = 1
POINTS_PER_LOSS = 0
POINTS_PER_TIE = 0.5


# basic rules of the game
def points(player, opponent):
    if player == opponent:
        return POINTS_PER_TIE
    if player == "rock" and opponent == "scissors":
        return POINTS_PER_WIN
    if player == "scissors" and opponent == "paper":
        return POINTS_PER_WIN
    if player == "paper" and opponent == "rock":
        return POINTS_PER_WIN
    return POINTS_PER_LOSS

# basic rules of the game
def beats_move(move):
    if move == "rock":
        return "paper"
    if move == "paper":
        return "scissors"
    if move == "scissors":
        return "rock"
    return None

# create the weighted probability vector
def biased_move(ind, bias):
    vec = [0.0, 0.0, 0.0]
    for (i, v) in enumerate(vec):
        if i == ind:
            vec[i] = 1 / 3 + bias
        else:
            vec[i] = 1 / 3 - bias / 2
    return vec

# determine how a player will play based on their last move and the last move of their opponent
def human_player(last_move, last_opponent_move, bias = 0.33):
    points_last = points(last_move, last_opponent_move)
    if points_last == 1: # player won last round
        ind = OPTIONS.index(last_move) # more likely to play the same move
    elif points_last == 0: # player lost last round
        ind = OPTIONS.index(beats_move(last_opponent_move)) # more likely to play the move that beats the opponent's last move
    else: # player tied last round
        ind = OPTIONS.index(last_move)
        bias = -bias / 2 # player is less likely to play the same move

    wv = biased_move(ind, bias)
    return random.choices(OPTIONS, weights = wv)[0]

def computer_player(last_move, last_opponent_move, M):
    ind_human = OPTIONS.index(last_opponent_move)
    ind_computer = OPTIONS.index(last_move)
    prob_vec = M[ind_human, ind_computer]
    return random.choices(OPTIONS, weights = prob_vec)[0]



def play_games(n_games, M):
    human_move = random.choice(OPTIONS)
    computer_move = random.choice(OPTIONS)
    score = 0
    for i in range(n_games):
        new_human_move = human_player(human_move, computer_move)
        new_computer_move = computer_player(computer_move, human_move, M)
        score += points(new_computer_move, new_human_move)
        human_move = new_human_move
        computer_move = new_computer_move

    return score / n_games

def updade_M(M, sd = 0.1):
    ind0 = random.choices(range(3))[0]
    ind1 = random.choices(range(3))[0]

    v = M[ind0, ind1]
    v = [x + np.random.normal(0, sd) for x in v] # this is what fixed the problem
    v = [max(0, min(1, x)) for x in v]
    v = [x / sum(v) for x in v]

    M[ind0, ind1] = v

    return M

M = np.random.uniform(0, 1, (3, 3, 3))
M = M / np.sum(M, axis = 2)[:, :, None]

games_per_round = 10000

p_win_original = play_games(games_per_round, M)
print(f"Original: {p_win_original}")
iters = 1000
p_win_old = p_win_original
M_old = M
for i in range(iters):
    M = M_old.copy()
    M = updade_M(M)
    # p_win_old = play_games(games_per_round, M_old)
    p_win_new = play_games(games_per_round, M)
    if p_win_new > p_win_old:
        M_old = M.copy()
        p_win_old = p_win_new
    else:
        # M = updade_M(M_old)
        pass


        # M = M_old
    if (i + 1) % 100 == 0:
        print(f"Iteration {i + 1}: {p_win_old}")


p_win_new = play_games(100000, M)
print(f"Final: {p_win_new}")

Original: 0.51375
Iteration 100: 0.5515
Iteration 200: 0.61695
Iteration 300: 0.625
Iteration 400: 0.6351
Iteration 500: 0.65145
Iteration 600: 0.659
Iteration 700: 0.659
Iteration 800: 0.66965
Iteration 900: 0.67785
Iteration 1000: 0.68695
Final: 0.6743


In [119]:
M

array([[[0.14369472, 0.        , 0.85630528],
        [1.        , 0.        , 0.        ],
        [0.4358606 , 0.47186204, 0.09227736]],

       [[0.        , 0.64388997, 0.35611003],
        [0.61126806, 0.25401302, 0.13471891],
        [0.        , 1.        , 0.        ]],

       [[0.        , 0.        , 1.        ],
        [0.86533196, 0.13466804, 0.        ],
        [0.2422907 , 0.25029481, 0.50741448]]])